# ⚖️ Propensity Score Matching

처치 확률을 기반으로 매칭하여 인과효과를 추정하는 방법입니다.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm

df = pd.read_csv("../data/simulated_health_data.csv")
X = df[["age", "sex", "hypertension"]]
y = df["treatment"]
ps_model = LogisticRegression().fit(X, y)
df["pscore"] = ps_model.predict_proba(X)[:, 1]

treated = df[df["treatment"] == 1]
control = df[df["treatment"] == 0]
nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[["pscore"]])
_, idx = nn.kneighbors(treated[["pscore"]])
matched = pd.concat([treated.reset_index(drop=True), control.iloc[idx.flatten()].reset_index(drop=True)])
model = sm.OLS(matched["outcome"], sm.add_constant(matched["treatment"])).fit()
model.summary()